## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

In [2]:
# загрузка данных
insurance = pd.read_csv('insurance.csv')

In [3]:
# просмотр первых 5-ти наблюдений
insurance.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
# просмотр общей информации о массиве данных
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


- имеем информацию по 5000 клиентам. Каждый индивид характеризуется 5-ю показателями: пол, возраст, зарплата, член, семьи и страховые выплаты. Страховые выплаты выступает в качестве целевого признака.

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Имеем 5000 наблюдений и 4 признака. Тем самым, размерность матрицы признаков $X$ составляет 5000х4. Для выполнения матричного умножения, размерность матрицы преобразования $P$ соответствует 4х4. Размерность новой матрицы признаков $XP$ составляет 5000х4.\
Далее, сформулируем условие первого порядка задачи минимизации квадрата ошибок:
$$
\frac{\partial \Big(y - (XP)w \Big)^2}{\partial w} = 2 \Big(y-(XP)w \Big)(-XP)^T=0.
$$

Сокращая $-2$ в вышеприведенной формуле, получим:
$$
\Big(y - (XP) w \Big)(XP)^T=0
$$
Далее, перенесем $ (XP)^T (XP) w $ в правую часть:
$$
(XP)^T y = (XP)^T (XP) w
$$
Выразим $w$:
$$
w =\Big((XP)^T (XP) \Big)^{-1} (XP)^T y = \Big(P^T X^T XP \Big)^{-1} P^T X^T y
$$
Далее, для получения предсказаний умножим новую матрицу признаков $XP$ на новый результирующий параметр $w$:
$$
a_{XP} = XP \Big( w \Big) = XP \Big( \big(P^T X^T XP \big)^{-1} P^T X^T y \Big) = 
XP \Big( P^{-1} X^{-1} (X^T)^{-1} (P^T)^{-1} P^T X^T y  \Big) =  
X P P^{-1} X^{-1} (X^T)^{-1} (P^T)^{-1} P^T X^T y
$$
Так как матрица $P$ квадратная, то $(P^T)^{-1}=(P^{-1})^T$.\
Также, примем во внимание следующие равенства:
$$
(XP)^{-1}=P^{-1} X^{-1}; (XP)^T=P^T X^T; PP^{-1}=I,
$$
где $I-$ матрица с единицами. В итоге, получим:
$
a_{XP}=
X \big(P P^{-1} \big) X^{-1} (X^T)^{-1} \big( P P^{-1} \big)^T X^T y
= X \big(I \big) \big( X^T X \big)^{-1} \big( I \big)^T X^T y
= X \big( X^T X \big)^{-1} X^T y
$.\
Сравним с исходной задачей:
$$
a = X (X^T X)^{-1} X^T y 
$$
Тем самым, умножение признаков $X$ на матрицу преобразования $P$ не меняет предсказания модели линейной регрессии.

## 3. Алгоритм преобразования

**Алгоритм**\
В качестве алгоритма преобразования можно предложить матрицу со случайными величинами, распределенными по нормальному закону $\xi \sim N(0, \sigma^{2})$, у которой существует обратная к ней матрица. Для этого, можно воспользоваться функцией `numpy.random.randn`.

**Обоснование**\
Выбор данной матрицы преобразования со случайными величинами $\xi \sim N(0, \sigma^{2})$ может объясняться тем, что при увеличении размера выборочной совокупности, статистические наблюдения достаточно хорошо описываются распределением Гаусса. Для проверки обратимости матрицы воспользуемся функцией `np.linalg.inv()`.\
Для обоснования неизменности качества линейной регрессии, вспомним функциональную форму нашей метрики, коэффициента детерминации:
$$
R^2 = \frac{\sum_{i=1}^{N} (a_{i}-\bar{y})^2}{\sum_{i=1}^{N} (y_{i}-\bar{y})^2}=
\frac{ (a-\bar{y})^T (a-\bar{y}) }{ (y-\bar{y})^T (y-\bar{y}) }=
\frac{ (Xw-\bar{y})^T (Xw-\bar{y}) }{ (y-\bar{y})^T (y-\bar{y}) },
$$
где $a_{i}, y_{i}-$ прогнозные, фактические величины целевого признака, соответственно; $\bar{y}-$ среднее значение целевого признака.\
Ранее, было выявлено что преобразование матрицы признаков не меняет вектор прогнозных значений $a=Xw=XPw, \forall P$.\
Тем самым, имеет место равенство коэффициентов детерминации для исходной и преобразованной матриц признаков:
$$
R^2_{X} = \frac{ (Xw-\bar{y})^T (Xw-\bar{y}) }{ (y-\bar{y})^T (y-\bar{y}) }=
\frac{ (XPw-\bar{y})^T (XPw-\bar{y}) }{ (y-\bar{y})^T (y-\bar{y}) }=R^2_{XP}.
$$


## 4. Проверка алгоритма

In [5]:
# выделим целевой признак
target_label = 'Страховые выплаты'
target = insurance[target_label]
# выделим остальные признаки индивидов
features = insurance.drop(labels=target_label, axis=1)

In [6]:
# сгенерируем матрицу преобразования
randn_matrix = np.random.randn(features.shape[1], features.shape[1])
# при вычислении обратной матрицы преобразования воспользуемся исключениями
try:
    inv_randn_matrix = np.linalg.inv(randn_matrix)
except:
    print('Матрица преобразования является необратимой (не имеет обратную матрицу)')
else:
    print('Вычисление обратной матрицы преобразования inv_randn_matrix выполнено')

Вычисление обратной матрицы преобразования inv_randn_matrix выполнено


Таким образом, подтверждается целесообразность использования выбранной матрицы преобразования.

In [7]:
# Через матричное умножение исходной матрицы признаков Х на матрицу преобразования Р
# сформируем новую матрицу признаков XP
new_features = features.values.dot(randn_matrix)

In [8]:
# Определим функцию, которая принимает матрицу признаков и вектор целевых значений, а
# на выходе получаем коэффиициент детерминации по модели линейной регрессии
def train_lin_model(features, target, test_size=0.25):
    '''
    featurez: матрица признаков
    target: вектор целевых значений
    test_size: процентная доля валидационного множества
    '''
    # поделим матрицу признаков и вектор целевых значений на обучающее и валидационное множества
    features_train, features_valid, target_train, target_valid =  train_test_split(features,
                                                                                   target,
                                                                                   test_size=test_size,
                                                                                   random_state=42)
    # инициализируем модель линейной регрессии
    model = LinearRegression(n_jobs=-1)
    # (1) обучим модель на тренировочном множестве,
    # (2) определим прогнозные значения на валидационном множестве
    predictions = model.fit(features_train, target_train).predict(features_valid)
    # вывод коэффициента детерминации
    return r2_score(target_valid, predictions)

In [9]:
# апробируем функцию train_lin_model на исходной матрице признаков
train_lin_model(features, target)

0.4254778540696311

In [10]:
# апробируем функцию train_lin_model на преобразованной матрице признаков
train_lin_model(new_features, target)

0.42547785406939875